In [ ]:
from common import *
from snobedo.snotel import CsvParser
use_hvplot()

In [ ]:
client = start_cluster(5, 12)
client_ip_and_port(client)

# Comparison between SNOTEL and iSNOBAL variables

In [ ]:
year = 2018
water_year = f"wy{year}"
time=slice(f"{year -1}-10-01", f"{year}-07-31")

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites_2x2.json')

## SNOTEL

In [ ]:
schofield_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Schofield/usda-csv/{year}-Schofield-Pass.csv',
)

butte_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Butte/usda-csv/{year}-Butte.csv',
)

taylor_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Taylor/usda-csv/{year}-Upper-Taylor.csv',
)

## iSnobal 

In [ ]:
wy_snow = xr.open_mfdataset(
    (SNOBAL_DIR / 'storm_day_fix' / f'{water_year}' / 'erw/*/snow.nc').as_posix(),
    parallel=True,
)

butte_snobal = wy_snow.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat).compute()
schofield_snobal = wy_snow.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat).compute()
taylor_snobal = wy_snow.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat).compute()

In [ ]:
client.shutdown()

In [ ]:
def max_swe_date(snotel_data):
    return snotel_data.loc[snotel_data['SWE(mm)'].idxmax()].name

In [ ]:
plot_data = {
    'Butte': {
        'snobal': butte_snobal,
        'snotel': butte_snotel_csv,
        'color_snobal': 'sandybrown',
        'color_snotel': 'skyblue',
        'marker_aso': 'd',
        'max_swe_snotel': max_swe_date(butte_snotel_csv)
    },
    'Schofield Pass': {
        'snobal': schofield_snobal,
        'snotel': schofield_snotel_csv,
        'color_snobal': 'goldenrod',
        'color_snotel': 'steelblue',
        'marker_aso': 'o',
        'max_swe_snotel': max_swe_date(schofield_snotel_csv)
    },
    'Upper Taylor': {
        'snobal': taylor_snobal,
        'snotel': taylor_snotel_csv,
        'color_snobal': 'bisque',
        'color_snotel': 'royalblue',
        'marker_aso': 's',
        'max_swe_snotel': max_swe_date(taylor_snotel_csv)
    },
}

### ASO Snow Depth 

In [ ]:
for site in plot_data.keys():
    plot_data[site]['ASO'] = pd.read_csv(DATA_DIR / f'ASO-data/SNOTEL-sites/{site}.csv', delimiter=',', header=None, parse_dates=[0], index_col=0)

## Compare SNOTEL to iSnobal

In [ ]:
from bokeh.plotting import figure, output_file, show

In [ ]:
def aso_scatter(site, plot):
    df = plot_data[site]['ASO']
    dates = df[df.index.year == year].index.unique().values
    
    for day in dates:
        max_depth = df[df.index == day].max().values[0]
        min_depth = df[df.index == day].min().values[0]
        
        if max_depth >= 0 and min_depth >= 0:
            plot.segment(
                day, max_depth, 
                day, min_depth, 
                color="black",
                alpha=0.9,
                line_width=2,
                legend_label=site
            )
            plot.circle(
                day, 
                max_depth, 
                color='black',
                size=8,
                legend_label=site
            )
            plot.circle(
                day, 
                min_depth, 
                color='black',
                size=8,
                legend_label=site
            )

In [ ]:
p = figure(
    x_axis_type='datetime',
    width=1280, 
    height=500
)
p.xaxis.axis_label = f'Water Year {year}'
p.yaxis.axis_label = 'Snow Depth (m)'
p.xaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"


x_axis = np.arange(0,365)

for site in plot_data.keys():
    max_snobal = plot_data[site]['snobal'].coarsen(**COARSEN_OPTS).max().squeeze(['x', 'y'])
    x_time = max_snobal.time.values
    
    max_snobal = max_snobal['thickness'].values
    min_snobal = plot_data[site]['snobal'].coarsen(**COARSEN_OPTS).min().squeeze(['x', 'y'])['thickness'].values
    
    # Max Depth
    p.segment(
        x0=plot_data[site]['snotel'].loc[plot_data[site]['snotel']['Depth(m)'].idxmax()].name,
        x1=plot_data[site]['snotel'].loc[plot_data[site]['snotel']['Depth(m)'].idxmax()].name,
        y0=0, 
        y1=2.2,
        line_width=2, 
        color='indigo', 
        legend_label=site
    )

    p.varea(
        x=x_time,
        y1=max_snobal,
        y2=min_snobal,
        color=plot_data[site]['color_snobal'],
        alpha=0.8,
        legend_label=site,
    )
    
    aso_scatter(site, p)
    
    p.line(x_time, max_snobal, color='black', line_dash='dashed', legend_label=site)
    p.line(x_time, min_snobal, color='black', line_dash='dashed', legend_label=site)

    
    p.line(
        x_time,
        plot_data[site]['snotel']['Depth(m)'],
        legend_label=site, 
        color=plot_data[site]['color_snotel'], 
        alpha=0.8, line_width=1.5,
    )
     
p.legend.click_policy="hide"
p.legend.glyph_width = 0

output_file('Snotel-Depth.html', 'Snow Depth Comparison', 'inline')
show(p)

In [ ]:
!mv Snotel-Depth.html ~/public_html